In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

In [2]:
#sign in

import wrds
db=wrds.Connection(wrds_username='hienle')
#db.create_pgpass_file()

Loading library list...
Done


### Data

The following cell contains code borrowed from the solution to the previous problem. It was required that we keep also the stocks that have been traded only during a limited part of the sample period.

In [4]:
# NO NEED TO RUN THIS

# get risk-free rate see http://www.crsp.com/files/treasury_guide_0.pdf
Rf = db.raw_sql("select  mcaldt,tmytm "
           "from crsp.tfz_mth_rf "           
            "where kytreasnox = 2000001 "
           "and mcaldt>='1970-01-01'"
            "and mcaldt<='2019-12-31'", date_cols=['mcaldt'])

#transform annualized and continuously compounded returns into simple and monthly returns:
Rf['tmytm'] = np.exp(Rf['tmytm']/12/100) - 1
Rf = Rf.rename(columns={ "mcaldt": "date","tmytm": "rf"})

#use own index instead
# get index returns
data_index=db.raw_sql("select  date,vwretd "
           "from crsp.msi "
           "where date>='1970-01-01'"
            "and date<='2019-12-31'", date_cols=['date'])


# get stock returns
crsp_m = db.raw_sql("""
                      select a.permno, a.date, 
                      b.shrcd, b.exchcd,
                      a.ret, a.shrout, a.prc
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1970' and '12/31/2019'
                      and b.exchcd between 1 and 2
                      and b.shrcd between 10 and 11
                      """, date_cols=['date']) 
msf=crsp_m.drop(['shrcd','exchcd'],axis=1)
del(crsp_m) 

msf['prc']=np.abs(msf['prc'])


#variables
msf['permno'] = msf['permno'].astype(int)
msf['size'] = msf['shrout'] * msf['prc'].abs()
msf['const'] = 1
msf.sort_values(['permno','date'])

msf['size_lag']= msf.groupby('permno')['size'].shift(1).to_frame()

obs=msf[['const','permno']].groupby(['permno']).sum().reset_index().rename(columns={'const': 'obs'})
msf = pd.merge(msf, obs, how='left', left_on=['permno'],right_on=['permno'])

In [6]:
# NO NEED TO RUN THIS EITHER

#################################
# Compute excess returns
#################################
data_index = pd.merge(data_index, Rf, how='left', left_on=['date'],right_on='date')
data_index['mprem']=(data_index['vwretd']-data_index['rf'])
data = pd.merge(msf, data_index, how='left', left_on=['date'],right_on='date')
data['exret']=data['ret']-data['rf']

print('duplicates in data on returns?')
print(data.duplicated(subset=['date','permno']).sum(),'\n')

print(' How many obervations do we have in a month?')
print('max:', max(data[['date','const']].groupby('date').sum()['const'].values))
print('min:', min(data[['date','const']].groupby('date').sum()['const'].values))

print('\n How often do we observe a given stock?')
print('max:', max(data['obs'].values))
print('min:', min(data['obs'].values))

duplicates in data on returns?
0 

 How many obervations do we have in a month?
max: 2532
min: 1403

 How often do we observe a given stock?
max: 600
min: 1


In [4]:
# data.to_csv("data_1970-2019.csv")
data = pd.read_csv("data_1970-2019.csv", index_col=0)
ff_3f = pd.read_csv("F-F_Research_Data_Factors.CSV", index_col=0)
ff_mom = pd.read_csv("F-F_Momentum_Factor.CSV", index_col=0)

ff = pd.merge(ff_3f, ff_mom, right_index=True, left_index=True)
ff = ff[(ff.index > 197000) & (ff.index <= 201912)]
ff.index = ff.index.astype(str).str[:4] + "-" + ff.index.astype(str).str[4:7]
ff.columns = ff.columns.str.strip(" ") # somehow the Mom column got weird whitespace at the end

/home/hienle/.virtualenvs/notebook/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### b,

For each stock and each month t, calculate the average return of the stock between
month t-12 and t-2. Then, in each month, sort the stocks into 10 deciles based
on that average return. Compute the value-weighted returns on those portfolios.
Compute the returns on a zero-cost portfolio that goes long in the group with the
highest past returns and short in the group with the lowest past returns. Compute
the alpha of this strategy with respect to the market as well as the SMB and HML
factors. What do you observe?

In [7]:
all_stocks = data.permno.unique()
all_stocks.shape # checking

(7979,)

In [32]:
all_stocks

array([10001, 10006, 10014, ..., 93422, 93423, 93426])

##### For each stock and each month t, calculate the average return of the stock between month t-12 and t-2.

In [5]:
# TAKES A LONG TIME => DON'T RUN

# """
stock_avg_ret_df = pd.DataFrame(columns=data.columns)

for s in data.permno.unique():
    temp_df = data[data.permno == s].sort_values(by="date").reset_index()
    temp_df["avg_ret"] = temp_df["exret"].shift(2).rolling(window=11).mean()
    stock_avg_ret_df = pd.concat([stock_avg_ret_df, temp_df])
    
# """

In [6]:
# SAVING TO LOCAL FOLDER

stock_avg_ret_df.to_csv("avg_ret_added.csv")

In [7]:
stock_avg_ret_df = pd.read_csv("avg_ret_added.csv", index_col=0)
stock_avg_ret_df["month"] = stock_avg_ret_df["date"].str[:4] + "-" + stock_avg_ret_df["date"].str[5:7]
stock_avg_ret_df = stock_avg_ret_df.set_index(["month", "permno"]).sort_index()
stock_avg_ret_df = stock_avg_ret_df[stock_avg_ret_df["avg_ret"].notna()]

/home/hienle/.virtualenvs/notebook/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


##### Then, in each month, sort the stocks into 10 deciles based on that average return (this is done in the loop). Compute the value-weighted returns on those portfolios.

In [8]:
# function to calculate value weighted return
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan

In [9]:
# Then, in each month, sort the stocks into 10 deciles based
# on that average return

# value weighted
months = sorted(list(set([i for (i,j) in stock_avg_ret_df.index])))

vw_deciles = pd.DataFrame(index=months, columns=[f"portfolio_{i}" for i in range(1,11)])


for month in months:
    temp = stock_avg_ret_df.loc[month]
    deciles = pd.DataFrame(pd.qcut(temp.avg_ret, 10)).dropna()
    lst_portfolios = deciles.reset_index().groupby(["avg_ret"])["permno"].apply(list)
    for i in range(1,11):
        index = lst_portfolios.index[i-1]
        stock_int = list(lst_portfolios.loc[index])[0]
        relevant_stocks = temp.loc[stock_int]
        
        vw_deciles.at[month, f"portfolio_{i}"] = wavg(relevant_stocks, "ret", "size_lag")        

In [10]:
vw_deciles.head()

,portfolio_1,portfolio_2,portfolio_3,portfolio_4,portfolio_5,portfolio_6,portfolio_7,portfolio_8,portfolio_9,portfolio_10
1971-01,0.101695,0.188679,0.15625,0.120312,0.0207254,-0.0696721,0.0438356,0.0610063,0.0327869,-0.0280899
1971-02,NaN,-0.0666667,-0.104895,-0.0818182,0.016129,-0.00577428,0,0.164467,-0.00529101,0.101734
1971-03,-0.027027,0.0517241,-0.0479705,0.039604,0.0909091,0.0690691,0.0144681,0.0942249,-0.0529101,0.157823
1971-04,-0.0277778,-0.00819672,0.0930233,0.161765,-0.0831461,-0.0585106,0.0351759,0.145251,0.122642,-0.0116959
1971-05,-0.114286,-0.0287037,-0.0420168,-0.0510549,-0.0423729,-0.0458537,0.0310924,-0.00712329,-0.102102,0.037037


#####  Compute the alpha of this strategy with respect to the market as well as the SMB and HML factors.

In [11]:
import statsmodels.api as sm

In [12]:
# compute alpha
factors_of_interest = ["Mkt-RF", "SMB", "HML"]
alpha_df = pd.DataFrame()

for i in range(1,11):
    y = vw_deciles[f"portfolio_{i}"].values
    for j in range(len(factors_of_interest)):
        x = ff.loc[vw_deciles.index][factors_of_interest[j]].values
        X = sm.add_constant(x)
        reg = sm.OLS(endog=y,exog=X,missing='drop')
        results = reg.fit()
        alpha_df.at[i, f"alpha_{factors_of_interest[j]}"] = results.params[0]
    
    for combo in itertools.combinations(factors_of_interest, r=2):
        combo = list(combo)
        x = ff.loc[vw_deciles.index][combo].values
        X = sm.add_constant(x)
        reg = sm.OLS(endog=y,exog=X,missing='drop')
        results = reg.fit()
        alpha_df.at[i, f"alpha_{'-'.join(combo)}"] = results.params[0]
    
    x_all = ff.loc[vw_deciles.index][factors_of_interest].values # all SMB, HML, Market Exposure
    X_all = sm.add_constant(x_all)
    reg_all = sm.OLS(endog=y, exog=X_all, missing="drop")
    results_all = reg_all.fit()
    alpha_df.at[i, f"alpha_all"] = results_all.params[0]

In [13]:
print(alpha_df)

    alpha_Mkt-RF  alpha_SMB  alpha_HML  alpha_Mkt-RF-SMB  alpha_Mkt-RF-HML  \
1      -0.002826   0.001619   0.004170         -0.003195         -0.003743   
2       0.005405   0.009774   0.011463          0.005186          0.004004   
3       0.002498   0.007043   0.008316          0.002270          0.000641   
4       0.004735   0.008987   0.009916          0.004678          0.003660   
5      -0.001327   0.003144   0.003722         -0.001393         -0.003069   
6       0.014182   0.018903   0.019573          0.014104          0.012399   
7       0.006395   0.011997   0.013334          0.006330          0.005236   
8       0.012368   0.017178   0.018186          0.012231          0.010618   
9       0.006135   0.011147   0.013779          0.005971          0.006440   
10      0.013150   0.017466   0.021277          0.012840          0.014482   

    alpha_SMB-HML  alpha_all  
1        0.001513  -0.004612  
2        0.009204   0.003347  
3        0.006305   0.000060  
4        0.008946

##### ...Compute the returns on a zero-cost portfolio that goes long in the group with the highest past returns and short in the group with the lowest past returns.

In [14]:
zc_deciles = pd.DataFrame(index=months, columns=["zc_portfolio"])


for month in months: # just copied from the previous cell 
    temp = stock_avg_ret_df.loc[month]
    deciles = pd.DataFrame(pd.qcut(temp.avg_ret, 10)).dropna()
    lst_portfolios = deciles.reset_index().groupby(["avg_ret"])["permno"].apply(list).sort_index()
    
    low_ind = lst_portfolios.loc[lst_portfolios.index[0]]   
    high_ind = lst_portfolios.loc[lst_portfolios.index[-1]]
    
    bad_stocks = temp.loc[low_ind] # short
    good_stocks = temp.loc[high_ind] # long
        
    zc_deciles.at[month, "zc_portfolio"] = sum(good_stocks.exret) - sum(bad_stocks.exret) 

#####  Compute the alpha of this strategy with respect to the market as well as the SMB and HML factors.

In [15]:
alpha_zc = {}

y_zc = zc_deciles["zc_portfolio"].values

for j in range(len(factors_of_interest)):
    x = ff.loc[zc_deciles.index][factors_of_interest[j]].values
    X = sm.add_constant(x)
    reg = sm.OLS(endog=y_zc,exog=X,missing='drop')
    results = reg.fit()
    alpha_zc[f"alpha_{factors_of_interest[j]}"] = results.params[0]

for combo in itertools.combinations(factors_of_interest, r=2):
    combo = list(combo)
    x = ff.loc[zc_deciles.index][combo].values
    X = sm.add_constant(x)
    reg = sm.OLS(endog=y_zc,exog=X,missing='drop')
    results = reg.fit()
    alpha_zc[f"alpha_{'-'.join(combo)}"] = results.params[0]

reg_all = sm.OLS(endog=y_zc, exog=X_all, missing="drop")
results_all = reg_all.fit()
alpha_zc[f"alpha_all"] = results_all.params[0]

alpha_zc

{'alpha_HML': 1.9233903007291226,
 'alpha_Mkt-RF': 1.9530243461368868,
 'alpha_Mkt-RF-HML': 2.299003197416491,
 'alpha_Mkt-RF-SMB': 1.9495179014517015,
 'alpha_SMB': 1.7934547800577918,
 'alpha_SMB-HML': 2.048623185659035,
 'alpha_all': 2.33922007692896}

##### COMMENTS:


The $\alpha$s (single-factor and 2- and 3-factor benchmark) for the value weighted models stayed close to 0 across the 10 portfolios, whereas $\alpha$s are very high in the case of the zero-cost portfolio, showing that this strategy outsmarts the benchmarks by a very large margin.

### c,
Sort the stocks in each month into two groups based on the previous month’s
market capitalization (such that the stocks in one group are larger than in the
other group). Repeat question b) for the two groups. When computing the alphas
for the long-short strategies, also control for the momentum factor from Kenneth
French’s website. What do you observe?

In [146]:
# ALSO TAKES VERY LONG TO RUN - DON'T RUN EITHER

"""
mk_capt_df = pd.DataFrame(columns=data.columns)
for stock in data.permno.unique():
    temp_df = data[data.permno == stock].sort_values(by="date").reset_index()
    temp_df["mk_capt"] = temp_df.shift(1)["prc"] * temp_df.shift(1)["shrout"] # previous month's mk capt
    mk_capt_df = pd.concat([mk_capt_df, temp_df])
mk_capt_df.to_csv("mk_capt_added.csv")
"""

In [164]:
stock_avg_ret_df.head()

date       ret   shrout     prc       size  const  \
month   permno                                                            
1971-01 10006   1971-01-29  0.043836   5614.0  47.625  267366.75      1   
        10014   1971-01-29  0.156250   4622.0   4.625   21376.75      1   
        10057   1971-01-29  0.188679   3842.0  15.750   60511.50      1   
        10102   1971-01-29 -0.028090  11348.0  21.625  245400.50      1   
        10137   1971-01-29  0.032787  20792.0  23.625  491211.00      1   

                 size_lag  obs   vwretd        rf     mprem     exret   index  \
month   permno                                                                  
1971-01 10006   256138.75  173  0.05203  0.003391  0.048639  0.040445   104.0   
        10014    18488.00   87  0.05203  0.003391  0.048639  0.152859   309.0   
        10057    50906.50  318  0.05203  0.003391  0.048639  0.185288  1178.0   
        10102   252493.00  100  0.05203  0.003391  0.048639 -0.031481  1911.0   
        10137   475617.00  493  0.05203  0.003391  0.048639  0.029396  2809.0   

                 avg_ret  
month   permno            
1971-01 10006  -0.000618  
        10014  -0.030625  
        10057  -0.045472  
        10102   0.022760  
        10137   0.012174

In [16]:
mk_capt_df = pd.read_csv("mk_capt_added.csv", index_col=0)
mk_capt_df["month"] = mk_capt_df["date"].str[:4] + "-" + mk_capt_df["date"].str[5:7]
mk_capt_df = mk_capt_df.set_index(["month", "permno"]).sort_index()
mk_capt_df = mk_capt_df.merge(stock_avg_ret_df[["avg_ret"]], left_index=True, right_index=True)

/home/hienle/.virtualenvs/notebook/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [17]:
mk_capt_df.head(10)

date       ret   shrout     prc         size  const  \
month   permno                                                              
1971-01 10006   1971-01-29  0.043836   5614.0  47.625   267366.750      1   
        10014   1971-01-29  0.156250   4622.0   4.625    21376.750      1   
        10057   1971-01-29  0.188679   3842.0  15.750    60511.500      1   
        10102   1971-01-29 -0.028090  11348.0  21.625   245400.500      1   
        10137   1971-01-29  0.032787  20792.0  23.625   491211.000      1   
        10145   1971-01-29  0.020725  27555.0  24.625   678541.875      1   
        10153   1971-01-29  0.120312  10690.0  17.875   191083.750      1   
        10161   1971-01-29 -0.014870  23652.0  33.125   783472.500      1   
        10188   1971-01-29  0.122449   4214.0  13.750    57942.500      1   
        10225   1971-01-29  0.024931  26733.0  46.250  1236401.250      1   

                   size_lag  obs   vwretd        rf     mprem     exret  \
month   permno                                                            
1971-01 10006    256138.750  173  0.05203  0.003391  0.048639  0.040445   
        10014     18488.000   87  0.05203  0.003391  0.048639  0.152859   
        10057     50906.500  318  0.05203  0.003391  0.048639  0.185288   
        10102    252493.000  100  0.05203  0.003391  0.048639 -0.031481   
        10137    475617.000  493  0.05203  0.003391  0.048639  0.029396   
        10145    664764.375  600  0.05203  0.003391  0.048639  0.017334   
        10153    171040.000  227  0.05203  0.003391  0.048639  0.116921   
        10161    795298.500  286  0.05203  0.003391  0.048639 -0.018261   
        10188     51621.500  102  0.05203  0.003391  0.048639  0.119058   
        10225   1206326.625  532  0.05203  0.003391  0.048639  0.021540   

                 index      mk_capt   avg_ret  
month   permno                                 
1971-01 10006    104.0   256138.750 -0.005688  
        10014    309.0    18488.000 -0.035695  
        10057   1178.0    50906.500 -0.050541  
        10102   1911.0   252493.000  0.017691  
        10137   2809.0   475617.000  0.007104  
        10145   3302.0   664764.375 -0.021418  
        10153   4803.0   171040.000 -0.029406  
        10161   4474.0   795298.500 -0.016657  
        10188   4855.0    51621.500 -0.049673  
        10225   5415.0  1206326.625  0.023557

##### Sort the stocks in each month into two groups based on the previous month’s market capitalization (such that the stocks in one group are larger than in the other group).

In [18]:
group1 = pd.DataFrame(columns=data.columns) # smaller stocks
group2 = pd.DataFrame(columns=data.columns) # larger stocks

for month in mk_capt_df.index.levels[0]:
    temp = mk_capt_df.loc[month].sort_values(by="mk_capt")
    temp = temp[temp['mk_capt'].notna()].reset_index()
    cutoff = int(temp.shape[0]/2)
    group1 = pd.concat([group1, temp.loc[:cutoff]])
    group2 = pd.concat([group2, temp.loc[cutoff:]])

In [19]:
# terrible code coming up
group1["month"] = group1["date"].str[:4] + "-" + group1["date"].str[5:7]
group2["month"] = group2["date"].str[:4] + "-" + group2["date"].str[5:7]

group1 = group1.set_index(["month", "permno"])
group2 = group2.set_index(["month", "permno"])

In [20]:
group1.head()

date       ret  shrout     prc       size const  \
month   permno                                                          
1971-01 30082   1971-01-29  0.229167   346.0  3.6875  1275.8750     1   
        34921   1971-01-29  0.127119   141.0  8.3125  1172.0625     1   
        35342   1971-01-29  0.750000   525.0  3.5000  1837.5000     1   
        37962   1971-01-29  0.250000   414.0  3.7500  1552.5000     1   
        41558   1971-01-29  0.343750   313.0  5.3750  1682.3750     1   

                size_lag  obs   vwretd        rf     mprem     exret  \
month   permno                                                         
1971-01 30082   1038.000  226  0.05203  0.003391  0.048639  0.225776   
        34921   1039.875  216  0.05203  0.003391  0.048639  0.123728   
        35342   1050.000   24  0.05203  0.003391  0.048639  0.746609   
        37962   1242.000   26  0.05203  0.003391  0.048639  0.246609   
        41558   1252.000  135  0.05203  0.003391  0.048639  0.340359   

                   index   mk_capt   avg_ret  
month   permno                                
1971-01 30082   322272.0  1038.000  0.001372  
        34921   378980.0  1039.875 -0.026343  
        35342   384322.0  1050.000 -0.080452  
        37962   411248.0  1242.000 -0.026553  
        41558   464983.0  1252.000 -0.042016

##### Repeat question b) for the two groups. When computing the alphas for the long-short strategies, also control for the momentum factor from Kenneth French’s website.

i. Sort the stocks into 10 deciles based on that average return (this is done in the loop). Compute the value-weighted returns on those portfolios, then compute the zero-cost portfolios.

In [21]:
def compute_vw_deciles(group, months):
    vw_deciles = pd.DataFrame(index=months)
    for month in months:
        temp = group.loc[month]
        deciles = pd.DataFrame(pd.qcut(temp.avg_ret, 10)).dropna()
        lst_portfolios = deciles.reset_index().groupby(["avg_ret"])["permno"].apply(list)

        for i in range(1,11):
            index = lst_portfolios.index[i-1]
            stock_int = list(lst_portfolios.loc[index])[0]
            relevant_stocks = temp.loc[stock_int]

            vw_deciles.at[month, f"portfolio_{i}"] = wavg(relevant_stocks, "ret", "size_lag")   
    return vw_deciles

def compute_zc_deciles(group, months):
    zc_deciles = pd.DataFrame(index=months, columns=["zc_portfolio"])


    for month in months: # just copied from the previous cell 
        temp = group.loc[month]
        deciles = pd.DataFrame(pd.qcut(temp.avg_ret, 10)).dropna()
        lst_portfolios = deciles.reset_index().groupby(["avg_ret"])["permno"].apply(list).sort_index()

        low_ind = lst_portfolios.loc[lst_portfolios.index[0]]   
        high_ind = lst_portfolios.loc[lst_portfolios.index[-1]]

        bad_stocks = temp.loc[low_ind] # short
        good_stocks = temp.loc[high_ind] # long

        zc_deciles.at[month, "zc_portfolio"] = sum(good_stocks.ret) - sum(bad_stocks.ret) 
    return zc_deciles

In [22]:
# value weighted for each group
months_g1 = sorted(list(set([i for (i,j) in group1.index])))
months_g2 = sorted(list(set([i for (i,j) in group2.index])))

vw_deciles_g1 = compute_vw_deciles(group1, months_g1)
vw_deciles_g2 = compute_vw_deciles(group2, months_g2)

zc_deciles_g1 = compute_zc_deciles(group1, months_g1)
zc_deciles_g2 = compute_zc_deciles(group2, months_g2)

ii. Compute alpha and also control for the momentum factor from Kenneth
French’s website when computing for long-short strategy

In [23]:
factors_vw = ["Mkt-RF", "SMB", "HML"]
factors_zc = ["Mkt-RF", "SMB", "HML", "Mom"]

def compute_alphas(vw_decs, zc_decs, group_name): # regress with all the elements required
    alpha_vw = pd.DataFrame()
    
    for i in range(1,11):
        y = vw_decs[f"portfolio_{i}"].values
        for j in range(len(factors_vw)):
            x = ff.loc[vw_decs.index][factors_vw[j]].values
            X = sm.add_constant(x)
            reg = sm.OLS(endog=y,exog=X,missing='drop')
            results = reg.fit()
            alpha_vw.at[i, f"alpha_{factors_vw[j]}"] = results.params[0]
        x_all = ff.loc[vw_decs.index][factors_vw].values # all SMB, HML, Market Exposure
        X_all = sm.add_constant(x_all)
        reg_all = sm.OLS(endog=y, exog=X_all, missing="drop")
        results_all = reg_all.fit()
        alpha_vw.at[i, f"alpha_all"] = results_all.params[0]

    alpha_zc = {}
    y_zc = zc_decs["zc_portfolio"].values
    for j in range(len(factors_zc)):
        x = ff.loc[zc_decs.index][factors_zc[j]].values
        X = sm.add_constant(x)
        reg = sm.OLS(endog=y_zc,exog=X,missing='drop')
        results = reg.fit()
        alpha_zc[f"alpha_{factors_zc[j]}"] = results.params[0]
    x_zc = ff.loc[zc_decs.index][factors_zc].values # all SMB, HML, Market Exposure and Momentum
    X_zc = sm.add_constant(x_zc)
    reg_zc = sm.OLS(endog=y_zc, exog=X_zc, missing="drop")
    results_zc = reg_zc.fit()
    alpha_zc["alpha_all"] = results_zc.params[0]    
    
    print(f"The alphas for the vw portfolios of {group_name} are: \n")
    print(alpha_vw)
    print(f"\n The alpha for the zero-cost portfolio of {group_name} is: \n")
    print(f"{alpha_zc}")

In [24]:
compute_alphas(vw_deciles_g1, zc_deciles_g1, "group 1")

The alphas for the vw portfolios of group 1 are: 

    alpha_Mkt-RF  alpha_SMB  alpha_HML  alpha_all
1       0.119745   0.122975   0.126872   0.116506
2       0.075319   0.077180   0.081538   0.074788
3       0.073540   0.077497   0.077930   0.069024
4       0.040464   0.043306   0.044593   0.035812
5       0.024932   0.028777   0.028879   0.019068
6       0.041652   0.046740   0.053856   0.042623
7       0.051981   0.054044   0.057773   0.045476
8       0.028111   0.032586   0.032978   0.024103
9       0.031529   0.036073   0.038527   0.027824
10      0.048213   0.049902   0.055469   0.045225

 The alpha for the zero-cost portfolio of group 1 is: 

{'alpha_Mkt-RF': 1.0923725643712499, 'alpha_SMB': 0.9780711428549024, 'alpha_HML': 1.0288139081357843, 'alpha_Mom': 0.1667105732747553, 'alpha_all': 0.13535133938508068}


In [25]:
compute_alphas(vw_deciles_g2, zc_deciles_g2, "group 2")

The alphas for the vw portfolios of group 2 are: 

    alpha_Mkt-RF  alpha_SMB  alpha_HML  alpha_all
1      -0.004808   0.003025   0.004740  -0.009289
2      -0.001171   0.003656   0.005468  -0.003296
3       0.003273   0.007436   0.009033   0.001102
4       0.000792   0.006739   0.007951  -0.002142
5       0.002342   0.006952   0.007129  -0.001375
6       0.002777   0.008903   0.010511  -0.000187
7       0.005791   0.010907   0.012996   0.004379
8       0.015373   0.019929   0.021566   0.013711
9       0.010965   0.016351   0.018875   0.009803
10      0.006985   0.012096   0.015005   0.006539

 The alpha for the zero-cost portfolio of group 2 is: 

{'alpha_Mkt-RF': 0.8905391095146492, 'alpha_SMB': 0.812879313843614, 'alpha_HML': 0.9540276948525068, 'alpha_Mom': 0.08462456970395457, 'alpha_all': 0.05141143232575765}


##### COMMENT:

In terms of the value-weighted portfolios, group 1 (smaller stocks) show consistently higher $\alpha$s across the portfolios and the factor benchmarks (1- and 3-factor) than those observed in group 2, where the $\alpha$s' values even show up negative in the full 4-factor benchmark model (with momentum). This shows that the smaller firms are doing better than the replicating benchmark portfolios, and that this is not the case with the bigger firms.